# **Machine_learning_models**

## **1 Import Libraries**





In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

In [ ]:
!pip install stop-words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp37-none-any.whl size=32917 sha256=55ca72f64503f4a0f2e1788860a15ebfee2c9d195f18266f4cdc1175149c14e3
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import time
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import gc
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

import math
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestRegressor
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from prettytable import PrettyTable
import pickle
import zipfile
from tqdm import tqdm
import os
from sklearn.linear_model import Ridge

from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from stop_words import get_stop_words
from collections import Counter

import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping,ModelCheckpoint
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense,Flatten
from tensorflow.keras.models import Model
from numpy import zeros
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model
tf.keras.backend.clear_session()

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### **1.1 Import data**

In [ ]:
PATH = 'My Drive/kì 2/học máy/mercari-price-suggestion-challenge/'

In [ ]:
X_train = pd.read_pickle('/content/gdrive/'+ PATH + 'pickles/X_train.pkl')
X_test = pd.read_pickle('/content/gdrive/'+ PATH + 'pickles/X_test.pkl')

In [ ]:
def read_pickle_dump(*args,filename = 'file'):
    argv = args
    variable_list = [i for i in argv]
    dir_name = "/content/gdrive/"+ PATH + "pickles/"
    filename_suffix = "pkl"
    file_path = os.path.join(dir_name, filename + "." + filename_suffix)
    with open(file_path,'rb') as f:
        variable_list = pickle.load(f)
    return variable_list

In [ ]:
tr_cat,te_cat= read_pickle_dump('tr_cat','te_cat',filename = "cat_features")

In [ ]:
def vectorizer(train,test,column,no_of_features,n_range,vector_type):
    '''
    Function to vectorize text using TFIDF/BoW
    '''
    if str(vector_type) == 'bow':
        vectorizer = CountVectorizer(ngram_range=n_range,max_features=no_of_features).fit(train[column])  #fitting
    else:
        vectorizer = TfidfVectorizer(ngram_range=n_range, max_features=no_of_features).fit(train[column]) # fit has to happen only on train data
    
    # we use the fitted vectorizer to convert the text to vector
    transformed_text = vectorizer.transform(tqdm(test[column]))
    print("After vectorizations")
    print(transformed_text.shape)
    print("="*100)
    return transformed_text

In [ ]:
X_train_bow_name = vectorizer(X_train,X_train,'name',100000,(1,2),'bow')
X_test_bow_name = vectorizer(X_train,X_test,'name',100000,(1,2),'bow')

100%|██████████| 1333494/1333494 [00:16<00:00, 79122.12it/s]


After vectorizations
(1333494, 100000)


100%|██████████| 148167/148167 [00:01<00:00, 78384.70it/s]


After vectorizations
(148167, 100000)


In [ ]:
X_train_tfidf_desc,X_test_tfidf_desc = read_pickle_dump('X_train_tfidf_desc','X_test_tfidf_desc',filename="tfidf_desc")

In [ ]:
x_train_set = hstack((X_train_bow_name,X_train_tfidf_desc,tr_cat,X_train['wc_name'].values.reshape(-1,1),X_train['wc_desc'].values.reshape(-1,1))).tocsr()
x_test_set = hstack((X_test_bow_name,X_test_tfidf_desc,te_cat,X_test['wc_name'].values.reshape(-1,1),X_test['wc_desc'].values.reshape(-1,1))).tocsr()

print("Final Data matrix")
print(x_train_set.shape)
print(x_test_set.shape)

Final Data matrix
(1333494, 206949)
(148167, 206949)


In [ ]:
def log_price(price):
    return np.log1p(price)

In [ ]:
global y_scalar
y_scalar = StandardScaler()
y_train = y_scalar.fit_transform(log_price(X_train['price']).values.reshape(-1, 1))
y_test = y_scalar.transform(log_price(X_test['price']).values.reshape(-1, 1))

### **1.2 Improve data**

In [ ]:
improved_data = pd.read_pickle('/content/gdrive/' + PATH + 'pickles/train_processed.pkl')

In [ ]:
from sklearn.model_selection import train_test_split
improved_train, improved_test= train_test_split(improved_data, test_size=0.10, random_state=123)

In [ ]:
improved_train.to_pickle('/content/gdrive/'+ PATH + 'pickles/improved_train.pkl')
improved_test.to_pickle('/content/gdrive/'+ PATH + 'pickles/improved_test.pkl')

In [ ]:
improved_train = pd.read_pickle('/content/gdrive/'+ PATH + 'pickles/improved_train.pkl')
improved_test = pd.read_pickle('/content/gdrive/'+ PATH + 'pickles/improved_test.pkl')

In [ ]:
improved_train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,log_prices,general_cat,subcat_1,subcat_2,desc_neg,desc_neu,desc_pos,is_expensive,scaler_name_len,scaler_desc_len
192736,192736,twd abraham mystery mini memoriam,2,Kids/Toys/Action Figures & Statues,funko,40.0,1,brand new amazing piece shipped care,3.688879,kids,toys,action figures statues,0.000,0.364,0.636,1,0.495447,-0.595547
940322,940322,toddler shoes,3,Kids/Girls 2T-5T/Shoes,unk_brand,9.0,0,nikes size 6c,2.197225,kids,girls 2t 5t,shoes,0.000,1.000,0.000,0,-1.395954,-0.739503
1388934,1388934,ann kline low dress pumps size 10 med,2,Women/Shoes/Pumps,anne klein,14.0,0,black leather black patent combine,2.639057,women,shoes,pumps,0.000,1.000,0.000,1,2.386847,-0.643533
982817,982817,velor boy sliders free shipping size 9m,2,Kids/Boys 0-24 Mos/One-Pieces,ralph lauren,10.0,1,no description yet,2.302585,kids,boys 0 24 mos,one pieces,0.524,0.476,0.000,1,1.756380,-0.739503
1009610,1009610,lularoe shirt,3,Women/Tops & Blouses/T-Shirts,lularoe,10.0,0,teal lularoe shirt good condition no free ship,2.302585,women,tops blouses,shirts,0.164,0.373,0.463,1,-1.395954,-0.499577


In [ ]:
def one_hot_encode(train,test):
    '''
    Function to one hot encode the categorical columns
    '''
    vectorizer = CountVectorizer(token_pattern='.+')
    # ta chỉ sử dụng hàm fit trên tậm dữ liệu train

    vectorizer = vectorizer.fit(train['category_name'].values)
    column_cat = vectorizer.transform(test['category_name'].values)

    vectorizer = vectorizer.fit(train['general_cat'].values)
    column_mc = vectorizer.transform(test['general_cat'].values)
    
    vectorizer = vectorizer.fit(train['subcat_1'].values)
    column_sb1 = vectorizer.transform(test['subcat_1'].values)
    
    vectorizer = vectorizer.fit(train['subcat_2'].values)
    column_sb2 = vectorizer.transform(test['subcat_2'].values)

    vectorizer = vectorizer.fit(train['brand_name'].astype(str))
    brand_encodes = vectorizer.transform(test['brand_name'].astype(str))
   
    print(column_cat.shape)
    print(column_mc.shape)
    print(column_sb1.shape)
    print(column_sb2.shape)
    print(brand_encodes.shape)
    print("="*100)
    return column_cat,column_mc,column_sb1,column_sb2,brand_encodes

In [ ]:
cat_ohe_tr,main_cat_ohe_tr,sub_cat_1_ohe_tr,sub_cat_2_ohe_tr,brand_ohe_tr = one_hot_encode(improved_train,improved_train)

(1302134, 1273)
(1302134, 10)
(1302134, 113)
(1302134, 855)
(1302134, 4669)


In [ ]:
cat_ohe_te,main_cat_ohe_te,sub_cat_1_ohe_te,sub_cat_2_ohe_te,brand_ohe_te = one_hot_encode(improved_train,improved_test)

(144682, 1273)
(144682, 10)
(144682, 113)
(144682, 855)
(144682, 4669)


In [ ]:
def get_dummies_item_id_shipping(df):
    df['item_condition_id'] = df["item_condition_id"].astype("category")
    df['shipping'] = df["shipping"].astype("category")
    item_id_shipping = csr_matrix(pd.get_dummies(df[['item_condition_id', 'shipping']],\
                                                      sparse=True).values)
    return item_id_shipping

In [ ]:
item_id_shipping_X_tr = get_dummies_item_id_shipping(improved_train)
item_id_shipping_X_te = get_dummies_item_id_shipping(improved_test)

In [ ]:
tr_cat2 = hstack((cat_ohe_tr,main_cat_ohe_tr,sub_cat_1_ohe_tr,sub_cat_2_ohe_tr,brand_ohe_tr,\
                     item_id_shipping_X_tr)).tocsr()
te_cat2 = hstack((cat_ohe_te,main_cat_ohe_te,sub_cat_1_ohe_te,sub_cat_2_ohe_te,brand_ohe_te,\
                     item_id_shipping_X_te)).tocsr()

print(tr_cat2.shape)
print(te_cat2.shape)

(1302134, 6927)
(144682, 6927)


In [ ]:
def pickle_dump(*args,filename = 'file'):
    argv = args
    list_variable = [i for i in argv]
    dir_name = "/content/gdrive/"+ PATH + "pickles/"
    filename_suffix = "pkl"
    file_path = os.path.join(dir_name, filename + "." + filename_suffix)
    with open(file_path, 'wb') as f:
        pickle.dump(list_variable, f)

In [ ]:
pickle_dump(tr_cat2,te_cat2,filename = "cat_features_2")

In [ ]:
def read_pickle_dump(*args,filename = 'file'):
    argv = args
    variable_list = [i for i in argv]
    dir_name = "/content/gdrive/"+ PATH + "pickles/"
    filename_suffix = "pkl"
    file_path = os.path.join(dir_name, filename + "." + filename_suffix)
    with open(file_path,'rb') as f:
        variable_list = pickle.load(f)
    return variable_list

In [ ]:
tr_cat2,te_cat2= read_pickle_dump('tr_cat2','te_cat2',filename = "cat_features_2")

In [ ]:
X_train_bow_name2 = vectorizer(improved_train,improved_train,'name',100000,(1,2),'bow')
X_test_bow_name2 = vectorizer(improved_train,improved_test,'name',100000,(1,2),'bow')

In [ ]:
X_train_tfidf_desc2 = vectorizer(improved_train,improved_train,'item_description',100000,(1,2),'tfidf')
X_test_tfidf_desc2 = vectorizer(improved_train,improved_test,'item_description',100000,(1,2),'tfidf')

In [ ]:
pickle_dump(X_train_tfidf_desc2,X_test_tfidf_desc2,filename="tfidf_desc2")

In [ ]:
X_train_tfidf_desc2,X_test_tfidf_desc2 = read_pickle_dump('X_train_tfidf_desc2','X_test_tfidf_desc2',filename="tfidf_desc2")

In [ ]:
x_train_set2 = hstack((X_train_bow_name2,X_train_tfidf_desc2,tr_cat2,\
                       improved_train['desc_neg'].values.reshape(-1,1),\
                       improved_train['desc_neu'].values.reshape(-1,1),\
                       improved_train['desc_pos'].values.reshape(-1,1),\
                       improved_train['is_expensive'].values.reshape(-1,1),\
                       improved_train['scaler_name_len'].values.reshape(-1,1),\
                       improved_train['scaler_desc_len'].values.reshape(-1,1))).tocsr()
x_test_set2 = hstack((X_test_bow_name2,X_test_tfidf_desc2,\
                      te_cat2,improved_test['desc_neg'].values.reshape(-1,1),\
                      improved_test['desc_neu'].values.reshape(-1,1),\
                      improved_test['desc_pos'].values.reshape(-1,1),\
                      improved_test['is_expensive'].values.reshape(-1,1),\
                      improved_test['scaler_name_len'].values.reshape(-1,1),\
                      improved_test['scaler_desc_len'].values.reshape(-1,1))).tocsr()

print("Final Data matrix")
print(x_train_set2.shape)
print(x_test_set2.shape)

NameError: ignored

In [ ]:
pickle_dump(x_train_set2,x_test_set2,filename="final_data_matrix")

In [ ]:
x_train_set2,x_test_set2 = read_pickle_dump('x_train_set2','x_test_set2',filename='final_data_matrix')
print("Final Data matrix")
print(x_train_set2.shape)
print(x_test_set2.shape)

Final Data matrix
(1302134, 206933)
(144682, 206933)


In [ ]:
global y_scalar
y_scalar = StandardScaler()
y_train2 = y_scalar.fit_transform(log_price(improved_train['price']).values.reshape(-1, 1))
y_test2 = y_scalar.transform(log_price(improved_test['price']).values.reshape(-1, 1))

## **5. Model**

In [ ]:
def hyperparameter_tuning_random(x,y,model_estimator,param_dict,cv_no):
    start = time.time()
    hyper_tuned = GridSearchCV(estimator = model_estimator, param_grid = param_dict,\
                                    return_train_score=True, scoring = 'neg_mean_squared_error',\
                                    cv = cv_no, \
                                    verbose=2, n_jobs = -1)
    hyper_tuned.fit(x,y)
    print("\n######################################################################\n")
    print ('Time taken for hyperparameter tuning is {} sec\n'.format(time.time()-start))
    print('The best parameters_: {}'.format(hyper_tuned.best_params_))
    return hyper_tuned.best_params_

In [ ]:
def rmsle_compute(y_true, y_pred):
    assert len(y_true) == len(y_pred)
    score = np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2)))
    return score

In [ ]:
def scale_back(x):
    '''
    Function to inverse transform the scaled values
    '''
    x= np.expm1(y_scalar.inverse_transform(x.reshape(-1,1))[:,0])#changes
    return x

**Mỗi mô hình thử nghiện với 2 tập dữ liệu khác nhau. Một tập là dữ liệu thô và tập còn lại đã qua xử lý dữ liệu**

###**5.1 Support vector regressor ver1**



In [ ]:
parameters_svr = {'alpha':[10**x for x in range(-4, 4)]}
model_svr = SGDRegressor(loss = "epsilon_insensitive",fit_intercept=False)

In [ ]:
best_parameters_svr = hyperparameter_tuning_random(x_train_set,y_train,model_svr,parameters_svr,3)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  8.3min finished



######################################################################

Time taken for hyperparameter tuning is 545.7301201820374 sec

The best parameters_: {'alpha': 0.0001}


**Model with tuned parameters:**

In [ ]:
model_svr_best_param = SGDRegressor(loss = "epsilon_insensitive",alpha = 0.0001, fit_intercept=False)
model_svr_best_param.fit(x_train_set,y_train[:,0])
y_train_pred_svr = model_svr_best_param.predict(x_train_set)
y_test_pred_svr = model_svr_best_param.predict(x_test_set)

**RMSLE Calculation:**

In [ ]:
rmsle_svr_reg_model_X_train = rmsle_compute(X_train['price'],scale_back(y_train_pred_svr))
print("RMSLE on train data: {}\n".format(rmsle_svr_reg_model_X_train))
rmsle_svr_reg_model_X_test = rmsle_compute(X_test['price'],scale_back(y_test_pred_svr))
print("RMSLE on test data: {}\n".format(rmsle_svr_reg_model_X_test))

###**5.2 Support vector regressor ver2**



In [ ]:
parameters_svr = {'alpha':[10**x for x in range(-4, 4)]}
model_svr = SGDRegressor(loss = "epsilon_insensitive",fit_intercept=False)

In [ ]:
best_parameters_svr = hyperparameter_tuning_random(x_train_set2,y_train2,model_svr,parameters_svr,3)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  2.0min finished



######################################################################

Time taken for hyperparameter tuning is 139.5860710144043 sec

The best parameters_: {'alpha': 0.0001}


**Model with tuned parameters:**

In [ ]:

model_svr_best_param = SGDRegressor(loss = "epsilon_insensitive",alpha = 0.0001, fit_intercept=False)
model_svr_best_param.fit(x_train_set2,y_train2[:,0])
y_train_pred_svr = model_svr_best_param.predict(x_train_set2)
y_test_pred_svr = model_svr_best_param.predict(x_test_set2)

**RMSLE Calculation:**

In [ ]:
rmsle_svr_reg_model_X_train = rmsle_compute(improved_train['price'],scale_back(y_train_pred_svr))
print("RMSLE on train data: {}\n".format(rmsle_svr_reg_model_X_train))
rmsle_svr_reg_model_X_test = rmsle_compute(improved_test['price'],scale_back(y_test_pred_svr))
print("RMSLE on test data: {}\n".format(rmsle_svr_reg_model_X_test))

### **5.3 RIDGE Regression ver1:**

**Hyperparameter tuning:**

In [ ]:
parameters_ridge = {'alpha':[0.0001,0.001,0.01,0.1,1.0,2.0,4.0,5.0,6.0]}
model_ridge = Ridge(
    solver='auto', fit_intercept=True,
    max_iter=100, normalize=False, tol=0.05, random_state = 1,
)

In [ ]:
rs_ridge = hyperparameter_tuning_random(x_train_set,y_train,model_ridge,parameters_ridge,3)

**Model with tuned parameters:**

In [ ]:
ridge_model = Ridge(
    solver='auto', fit_intercept=True,
    max_iter=100, normalize=False, tol=0.05, alpha=6.0,random_state = 1,
)
ridge_model.fit(x_train_set, y_train)

In [ ]:
preds_ridge_tr = ridge_model.predict(x_train_set)
preds_ridge = ridge_model.predict(x_test_set)

**RMSLE Calculation:**

In [ ]:

rmsle_ridge_X_train = rmsle_compute(X_train['price'],scale_back(preds_ridge_tr))
print("RMSLE on train data: {}\n".format(rmsle_ridge_X_train))
rmsle_ridge_X_test = rmsle_compute(X_test['price'],scale_back(preds_ridge))
print("RMSLE on test data: {}\n".format(rmsle_ridge_X_test))

**Ridge regression with RidgeCV**

In [ ]:

from sklearn.linear_model import RidgeCV
print("Fitting Ridge model on training examples...")
ridge_modelCV = RidgeCV(
    fit_intercept=True, alphas=[6.0],
    normalize=False, cv = 2, scoring='neg_mean_squared_error',
)
ridge_modelCV.fit(x_train_set, y_train)

In [ ]:
preds_ridgeCV_tr = ridge_modelCV.predict(x_train_set)
preds_ridgeCV = ridge_modelCV.predict(x_test_set)

**RMSLE Calculation:**

In [ ]:
rmsle_ridge_X_train = rmsle_compute(X_train['price'],scale_back(preds_ridgeCV_tr))
print("RMSLE on train data: {}\n".format(rmsle_ridge_X_train))
rmsle_ridge_X_test = rmsle_compute(X_test['price'],scale_back(preds_ridgeCV))
print("RMSLE on test data: {}\n".format(rmsle_ridge_X_test))

### **5.4 RIDGE Regression ver2:**

**Hyperparameter tuning:**

In [ ]:
parameters_ridge = {'alpha':[0.0001,0.001,0.01,0.1,1.0,2.0,4.0,5.0,6.0]}
model_ridge = Ridge(
    solver='auto', fit_intercept=True,
    max_iter=100, normalize=False, tol=0.05, random_state = 1,
)

In [ ]:
rs_ridge = hyperparameter_tuning_random(x_train_set2,y_train2,model_ridge,parameters_ridge,3)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:  3.8min finished



######################################################################

Time taken for hyperparameter tuning is 246.32548880577087 sec

The best parameters_: {'alpha': 0.1}


**Model with tuned parameters:**

In [ ]:
ridge_model = Ridge(
    solver='auto', fit_intercept=True,
    max_iter=100, normalize=False, tol=0.05, alpha=6.0,random_state = 1,
)
ridge_model.fit(x_train_set2, y_train2)

Ridge(alpha=6.0, copy_X=True, fit_intercept=True, max_iter=100, normalize=False,
      random_state=1, solver='auto', tol=0.05)

In [ ]:
preds_ridge_tr = ridge_model.predict(x_train_set2)
preds_ridge = ridge_model.predict(x_test_set2)

In [ ]:
pickle_dump(ridge_model,filename = "ridge_model")

**RMSLE Calculation:**

In [ ]:

rmsle_ridge_X_train = rmsle_compute(improved_train['price'],scale_back(preds_ridge_tr))
print("RMSLE on train data: {}\n".format(rmsle_ridge_X_train))
rmsle_ridge_X_test = rmsle_compute(improved_test['price'],scale_back(preds_ridge))
print("RMSLE on test data: {}\n".format(rmsle_ridge_X_test))

**Ridge regression with RidgeCV**

In [ ]:

from sklearn.linear_model import RidgeCV
print("Fitting Ridge model on training examples...")
ridge_modelCV = RidgeCV(
    fit_intercept=True, alphas=[0.1],
    normalize=False, cv = 2, scoring='neg_mean_squared_error',n
)
ridge_modelCV.fit(x_train_set2, y_train2)

Fitting Ridge model on training examples...


RidgeCV(alphas=array([0.1]), cv=2, fit_intercept=True, gcv_mode=None,
        normalize=False, scoring='neg_mean_squared_error',
        store_cv_values=False)

In [ ]:
preds_ridgeCV_tr = ridge_modelCV.predict(x_train_set2)
preds_ridgeCV = ridge_modelCV.predict(x_test_set2)

**RMSLE Calculation:**

In [ ]:
rmsle_ridge_X_train = rmsle_compute(improved_train['price'],scale_back(preds_ridgeCV_tr))
print("RMSLE on train data: {}\n".format(rmsle_ridge_X_train))
rmsle_ridge_X_test = rmsle_compute(improved_test['price'],scale_back(preds_ridgeCV))
print("RMSLE on test data: {}\n".format(rmsle_ridge_X_test))

RMSLE on train data: 0.37953629848325426

RMSLE on test data: 0.4460674689970452



**Sau khi thử nghiệm với 2 mô hình trên thì cho thấy kết quả được huấn luyện bởi tập dữ liệu đã qua xử lý tốt hơn hẳn nên từ đây ta sẽ xử dụng tập dữ liệu đã qua huấn luyện**

### **5.5 Lasso**

In [ ]:
from sklearn.linear_model import LassoCV

lasso_cv = LassoCV(alphas=(0.01, 0.1, 1.0, 10.0), cv=3)
lasso_cv.fit(x_train_set2, y_train2)
preds_ridge = ridge_model.predict(x_test_set2)
rmsl_error = rmsle_compute(improved_test['price'],scale_back(preds_ridge))
print("Optimal alpha:",lasso_cv.alpha_)
print("RMSLE:",rmsl_error)

In [ ]:
pickle_dump(lasso_cv,filename = "lasso_cv")

### **5.6 SGDR**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor
params = {"loss": ['squared_loss', 'epsilon_insensitive'],
          "alpha": [0.00001, 0.0001,0.001,0.01,0.1,0,1,10],
          "epsilon": [0.001, 0.01, 0.1, 0, 1, 10]
          }

grid_cv = GridSearchCV(SGDRegressor(learning_rate='adaptive'), cv=3, param_grid=params, scoring='neg_mean_squared_error', return_train_score=True, n_jobs=-1)
grid_cv.fit(x_train_set2, y_train2)

GridSearchCV(cv=3, error_score=nan,
             estimator=SGDRegressor(alpha=0.0001, average=False,
                                    early_stopping=False, epsilon=0.1,
                                    eta0=0.01, fit_intercept=True,
                                    l1_ratio=0.15, learning_rate='adaptive',
                                    loss='squared_loss', max_iter=1000,
                                    n_iter_no_change=5, penalty='l2',
                                    power_t=0.25, random_state=None,
                                    shuffle=True, tol=0.001,
                                    validation_fraction=0.1, verbose=0,
                                    warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.01, 0.1, 0, 1, 10],
                         'epsilon': [0.001, 0.01, 0.1, 0, 1, 10],
                         'loss': ['squared_loss', 'epsilon_insensitive']},
             pre_dispat

In [ ]:
preds_ridge = grid_cv.predict(x_test_set2)
rmsl_error = rmsle_compute(improved_test['price'],scale_back(preds_ridge))
print(rmsl_error)

0.4325177062631228


In [ ]:
pickle_dump(grid_cv,filename = "grid_cv")

### **5.7 LGBM**

In [ ]:
import lightgbm as lgb
from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

params = {
    "learning_rate": uniform(0.05,0.7),
    "max_depth": randint(2,10),
    "subsample": uniform(0.2,0.8),
    "colsample_bytree": uniform(0.2,0.8),
    "n_estimators": randint(80,300),
    "reg_alpha": [0.0001,0.001,0.01,0.1,1,10],
    "reg_lambda": [0.0001,0.001,0.01,0.1,1,10]
}

lgb_model = lgb.LGBMRegressor()
rand_cv = RandomizedSearchCV(lgb_model, param_distributions=params, cv=2, n_iter=100,\ 
                             scoring='neg_mean_squared_error', return_train_score=True, verbose=3, n_jobs=-1)
rand_cv.fit(x_train_set2, y_train2)

Fitting 2 folds for each of 100 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed: 79.3min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed: 338.5min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 520.9min finished


RandomizedSearchCV(cv=2, error_score=nan,
                   estimator=LGBMRegressor(boosting_type='gbdt',
                                           class_weight=None,
                                           colsample_bytree=1.0,
                                           importance_type='split',
                                           learning_rate=0.1, max_depth=-1,
                                           min_child_samples=20,
                                           min_child_weight=0.001,
                                           min_split_gain=0.0, n_estimators=100,
                                           n_jobs=-1, num_leaves=31,
                                           objective=None, random_state=None,
                                           reg_alpha=0.0, reg_lambda=0.0,
                                           silen...
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f30166a8950>,
        

In [ ]:
preds_ridge = rand_cv.predict(x_test_set2)
rmsl_error = rmsle_compute(improved_test['price'],scale_back(preds_ridge))
print(rmsl_error)

0.4618958763129155


In [ ]:
rand_cv.best_params_

{'colsample_bytree': 0.9531131832288895,
 'learning_rate': 0.6272545886347214,
 'max_depth': 8,
 'n_estimators': 259,
 'reg_alpha': 0.01,
 'reg_lambda': 1,
 'subsample': 0.5123801446840281}

In [ ]:
pickle_dump(rand_cv,filename = "rand_cv")